In [1]:
import numpy as np
import os, subprocess, ctypes
import win32gui, win32com.client, pyautogui, keyboard
import json, time
from PIL import Image

pyautogui.FAILSAFE = False

In [2]:
nexon_launch_target = "nxl://launch/10100"
steam_launch_target = "steam://rungameid/216150"

In [3]:
def get_maple_hwnd():
    return win32gui.FindWindowEx(None, None, None, 'MapleStory')

def focus_maple():
    windows = []
    hwnd = get_maple_hwnd()
    shell = win32com.client.Dispatch("WScript.Shell")
    shell.SendKeys('%')
    try:
        win32gui.SetForegroundWindow(hwnd)
        win32gui.ShowWindow(hwnd, 5)
        win32gui.BringWindowToTop(hwnd)
        win32gui.SetActiveWindow(hwnd)
    except: pass
    
def grab_maple(focus=True):
    maple_window = pyautogui.getWindowsWithTitle("MapleStory")[0]
    if focus: focus_maple()
    img = pyautogui.screenshot(region=maple_window.box)
    return img, maple_window.box
                                          
def check_admin():
    try:
        return os.getuid() == 0
    except AttributeError:
        return ctypes.windll.shell32.IsUserAnAdmin() != 0

def press_and_release(key, release_delay=.1, repeat=1):
    for i in range(repeat):
        keyboard.press(key)
        sleep(release_delay)
        keyboard.release(key)
        
def locate(img_path):
    box = pyautogui.locateOnScreen(img_path)
    return box

def locate_and_click(img_path, clicks=1, clicks_interval=0, duration=0):
    box = locate(img_path)
    if box:
        pyautogui.click(box, duration=duration, clicks=clicks, interval=clicks_interval)
        return True
    return False

def sleep(seconds):
    time.sleep(seconds)

In [4]:
class MapleController:
    def __init__(self, settings):
        if type(settings) == str:
            with open(settings) as fd:
                settings = json.load(fd)
                
        self.hp_potion_key = settings['hp_potion']
        self.mana_potion_key = settings['mana_potion']
        self.potions_per_use = settings.get('potions_per_use', 1)
        self.delay_per_potion = settings.get('delay_per_potion', 0)
        self.jump_key = settings['jump']
        self.right_key = settings['right']
        self.left_key = settings['left']
        self.up_key = settings['up']
        self.hold_up_state = settings['hold_up']
        self.pet_food_key = settings['pet_food']
        self.pet_food_period = settings['pet_food_period']
        self.direction_period = settings['change_direction_period']
        self.doublejump_state = settings['doublejump']
        self.channel_period =  settings['change_channel_period']
        self.attack_key = settings['attack']
        self.attack_interval = settings['attack_interval']
        self.attack_repeat = settings['attack_repeat']
        self.buffs_key = settings['buffs']
        self.buffs_period = settings['buffs_period']
        self.buffs_interval = settings['buffs_interval']
        self.skills = settings['skills']
        self.skills_interval = settings['skills_interval']
        self.login_skills = settings['login_skills']
        self.login_skills_interval = settings['login_skills_interval']
        self.install_origin = settings['install_origin']
        self.world = settings['world']
        self.PIC = settings['PIC']
        self.attempt_restart_after_crash_state = settings['attempt_restart_after_crash']

        self.cooldowns = {}
        
    def launch_maple(self):
        launch_target = steam_launch_target if self.install_origin == 'steam' else nexon_launch_target
        subprocess.run("start {}".format(launch_target), shell=True)
        
    def close_maple(self):
        os.system('taskkill /f /im MapleStory.exe')
        
    def select_world(self, world, channel=1):
        if self.check_world_menu_open():
            if not locate("refs/worlds/{}.png".format(world)):
                self.grab_frame()
                press_and_release('up')
                sleep(.25)
                press_and_release('up')
                sleep(.25)
            if locate_and_click("refs/worlds/{}.png".format(world), duration=1):
                sleep(.5)
                self.grab_frame()
                if self.check_world_menu_channels_menu_open():
                    for i in range(channel-1):
                        press_and_release('right')
                        sleep(.3)
                    press_and_release('enter')
                    sleep(1)
                else:
                    raise Exception('Faild selecting world {}, channels menu was not detected.'.format(world))
            else:
                raise Exception('Faild selecting world {}, world button was not detected.'.format(world))
        
    def enter_PIC(self, PIC):
        for key in PIC:
            if not locate_and_click('refs/PIC keys/{}.png'.format(key), duration=.5):
                raise Exception('Faild entering PIC, unable to find the key for `{}`.'.format(key))
        press_and_release('enter')
    
    def grab_frame(self):
        if self.get_status() == 1:
            self.frame_pil, self.box = grab_maple()
            self.frame_arr = np.asarray(self.frame_pil)
            return self.frame_pil, self.frame_arr, self.box
        else:
            raise Exception('Unable to grab a screenshot from maplestory because maplestory isn\'t running.')
        
    def check_health(self):
        a, b = np.array((470, 578))
        target_health = np.array([154, 152, 155])
        return np.all(self.frame_arr[b, a] == target_health)
    
    def check_health_and_heal(self, after_delay=0):
        if self.check_health():
            self.heal()
            sleep(after_delay)

    def check_mana(self):
        a, b = np.array((470, 594))
        target_mana = np.array([157, 155, 158])
        return np.all(self.frame_arr[b, a] == target_mana)
    
    def heal(self):
        for i in range(self.potions_per_use):
            press_and_release(self.hp_potion_key)
            if i > 0: sleep(self.delay_per_potion)
    
    def fill_mana(self, delay_per_fill=.25):
        for i in range(self.potions_per_use):
            press_and_release(self.mana_potion_key)
            if i > 0: sleep(self.delay_per_potion)
    
    def jump(self):
        press_and_release(self.jump_key)
    
    def doublejump(self):
        press_and_release(self.jump_key, repeat=2)

    def press_right(self):
        keyboard.press(self.right_key)

    def release_right(self):
        keyboard.release(self.right_key)

    def press_left(self):
        keyboard.press(self.left_key)

    def release_left(self):
        keyboard.release(self.left_key)
        
    def press_up(self):
        keyboard.press(self.up_key)

    def release_up(self):
        keyboard.release(self.up_key)
        
    def feed_pet(self):
        press_and_release(self.pet_food_key)
    
    def attack(self):
        for i in range(self.attack_repeat):
            press_and_release(self.attack_key)
            sleep(self.attack_interval)
            
    def check_world_menu_open(self):
        return True if pyautogui.locate("refs/messages/Select a world.png", self.frame_pil) else False

    def check_rune_message_open(self):
        return True if pyautogui.locate("refs/messages/Rune message 1.png", self.frame_pil) else False

    def check_settings_menu_open(self):
        return True if pyautogui.locate("refs/menus/Settings menu E.png", self.frame_pil) else False
    
    def check_ingame_channel_menu_open(self):
        return True if pyautogui.locate("refs/buttons/Change channel button.png", self.frame_pil) else False
    
    def check_unable_change_channel_dialog_open(self):
        return True if pyautogui.locate("refs/messages/Unable to change channel dialog.png", self.frame_pil) else False
    
    def check_news_window_open(self):
        return True if pyautogui.locate("refs/buttons/Login news buttons.png", self.frame_pil) else False
    
    def check_world_menu_channels_menu_open(self):
        return True if pyautogui.locate("refs/buttons/World menu channels button.png", self.frame_pil) else False
    
    def check_characters_menu_open(self):
        return True if pyautogui.locate("refs/buttons/Create character button.png", self.frame_pil) else False

    def check_cashshop_button_inview(self):
        return True if pyautogui.locate("refs/buttons/Cash shop button.png", self.frame_pil) else False
    
    def check_black_screen(self, x=100, y=100):
        self.grab_frame()
        return np.all(self.frame_arr[y, x] == [0, 0, 0])
        
    def move_channel(self):
        while self.get_status() == 1:
            # Make sure player wont die during channel change
            self.jump_or_doublejump()
            self.grab_frame()
            self.check_health_and_heal()

            press_and_release('esc')
            sleep(.15)

            self.grab_frame()
            if self.check_settings_menu_open():
                press_and_release('enter')
                sleep(.15)

                self.grab_frame()
                if self.check_ingame_channel_menu_open():
                    press_and_release('right')
                    sleep(.15)
                    
                    # Make sure player wont die during channel change
                    self.grab_frame()
                    if self.check_health():
                        press_and_release('esc')
                        self.attack()
                        self.heal()
                        sleep(.15)
                        continue
                        
                    press_and_release('enter')
                    sleep(1)
                    
                    self.grab_frame()
                    if self.check_unable_change_channel_dialog_open():
                        press_and_release('enter')
                        continue
                    
                    # Random pixel turns black must mean success
                    # Or if all screen turned black
                    px = self.frame_arr[100, 100]
                    if np.all(px == [0, 0, 0]): 
                        break
                        
    def jump_or_doublejump(self):
        if self.doublejump_state:
            self.doublejump()
        else:
            self.jump()
                        
    def use_buffs(self):
        for key in self.buffs_key:
            self.check_health_and_heal()
            sleep(self.buffs_interval)
            press_and_release(key)
            
    def restart_cooldown(self, key):
        self.cooldowns[key] = time.time()
        
    def check_cooldown(self, key, cooldown):
        if cooldown == -1:
            return False
        if key in self.cooldowns:
            return True if time.time() - self.cooldowns[key] > cooldown else False
        else:
            return True
 
    def use_skill(self, skill_settings):
        for i in range(5):
            keyboard.press(skill_settings['key'])
            sleep(skill_settings.get('hold', .1) / 5)
        keyboard.release(skill_settings['key'])
        
    def use_uncooldowned_skills(self):
        if any(self.check_cooldown(skill['key'], skill['period']) for skill in self.skills):
            sleep(.5)
            
        for skill in self.skills:
            self.check_health_and_heal()
            if self.check_cooldown(skill['key'], skill['period']):
                self.use_skill(skill)
                self.restart_cooldown(skill['key'])
                sleep(self.skills_interval)
                
    def use_login_skills(self):
        for skill in self.login_skills:
            self.use_skill(skill)
            sleep(self.login_skills_interval)

    def get_status(self):
        if get_maple_hwnd() == 0: return 0
        x = subprocess.check_output('tasklist /v /FI "IMAGENAME eq MapleStory.exe"')
        x = ([str(i) for i in x.splitlines()][3:])
        if len(x) < 1: return 0
        elif "Running" in x[0]: return 1
        elif "Not Responding" in x[0]: return 2